In [1]:
from __future__ import print_function, division
import cv2
import torch
import numpy as np
import pickle
import math

In [3]:
%load_ext autoreload
%autoreload 2

from featureBA.src.model_philipp import sparse3DBA
from featureBA.src.utils import sobel_filter

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [4]:
data = pickle.load(open("toy_example/data/toyexample_1_data.p", 'rb'))
img = cv2.imread("toy_example/data/toyexample_1.png")

In [5]:
T_perturbed = np.array([[math.cos(10*math.pi/180), -math.sin(10*math.pi/180), 0, 0],
             [math.sin(10*math.pi/180), math.cos(10*math.pi/180), 0, 0],
             [0, 0, 1, 0]])

In [6]:
data['coords'] = np.around(data['2d_points']).astype(int) - 1

In [7]:
P_perturbed = np.dot(data['K'], T_perturbed)
projected_2d = np.dot(P_perturbed, np.concatenate((data['3d_points'], np.ones(len(data['3d_points']))[:, None]),-1).T)
projected_2d = (projected_2d.T/projected_2d.T[:,2,None])[:, :2]

In [8]:
coords_2d = np.around(projected_2d)
coords_2d = coords_2d.astype(int) - 1

In [9]:
img = img.astype('uint8')
for i, p in enumerate(coords_2d):
    print(i, p)
    cv2.circle(img, tuple(p), 1, (128, 128, 0), -1)

0 [255 265]
1 [253 268]
2 [236 250]
3 [259 268]
4 [246 245]
5 [258 265]
6 [265 244]


In [10]:
cv2.imshow('image',img)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [399]:
cv2.imwrite('perturbed_points.png', img)

True

### run BA

In [400]:
%load_ext autoreload
%autoreload 2

from featureBA.src.model import sparse3DBA
from featureBA.src.utils import sobel_filter

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [401]:
img = cv2.imread("toy_example/data/toyexample_1.png", 0)
img = img.astype('double')

In [402]:
img_torch = torch.from_numpy(img)[None,...]
grad_x, grad_y = sobel_filter(img_torch)

In [403]:
img = grad_y.numpy().reshape(img.shape).astype('uint8')
for i, p in enumerate(coords_2d):
    print(i, p)
    cv2.circle(img, tuple(p), 1, (128, 128, 0), -1)
cv2.imshow('image', img)
cv2.waitKey(0)
cv2.destroyAllWindows()

0 [174 149]
1 [128 101]
2 [ 90 154]
3 [ 70 164]
4 [155 165]
5 [135 112]
6 [175 132]


In [404]:
cv2.imwrite("grad_x.png", grad_x.numpy().reshape(img.shape).astype('uint8'))
cv2.imwrite("grad_y.png", grad_y.numpy().reshape(img.shape).astype('uint8'))

True

In [405]:
pts3D = torch.from_numpy(data['3d_points'][:,:3])
ref2d = torch.from_numpy(np.flip(data['coords'], axis = 1).copy())
feature_ref = torch.cat([img_torch[:, i, j].unsqueeze(0) for i, j in zip(ref2d[:,0], ref2d[:,1])]).type(torch.DoubleTensor)
feature_map_query = img_torch.type(torch.DoubleTensor)
R_init, t_init = torch.from_numpy(T_perturbed[:, :3]), torch.from_numpy(T_perturbed[:, 3])
feature_grad_x = grad_x
feature_grad_y = grad_y
K = torch.from_numpy(data['K'])

In [406]:
from __future__ import print_function, division

from featureBA.src.utils import (from_homogeneous, to_homogeneous,
                                 batched_eye_like, skew_symmetric, so3exp_map)

from featureBA.src.utils import squared_loss, scaled_loss
import torch
from torch import nn
import numpy as np

def optimizer_step(g, H, lambda_=0):
    """One optimization step with Gauss-Newton or Levenberg-Marquardt.
    Args:
        g: batched gradient tensor of size (..., N).
        H: batched hessian tensor of size (..., N, N).
        lambda_: damping factor for LM (use GN if lambda_=0).
    """
    if lambda_:  # LM instead of GN
        D = (H.diagonal(dim1=-2, dim2=-1) + 1e-9).diag_embed()
        H = H + D*lambda_
    try:
        P = torch.inverse(H)
    except RuntimeError as e:
        logging.warning(f'Determinant: {torch.det(H)}')
        raise e
    delta = -(P @ g[..., None])[..., 0]
    return delta

def indexing_(feature_map, points):
    '''
    Function gives x and y gradients for 3D points in camera frame.

    inputs: (All pytorch tensors)
    @feature_map : x gradient of the feature map (CxHxW)
    @points : pixel coordinates of points (Nx2)

    outputs: 
    features : features for the points (NxC)
    '''

    features = torch.cat([feature_map[:, i, j].unsqueeze(0) for i, j in zip(points[:,0], points[:,1])])

    return features

In [673]:
n_iters = 100
loss_fn = squared_loss
lambda_ = 0.01

R = R_init
t = t_init

In [759]:
p_3d_1 = (torch.mm(R, pts3D.T).T + t)
p_proj_1 = torch.round(from_homogeneous(torch.mm(K, p_3d_1.T).T)).type(torch.IntTensor)-1
print(p_proj_1)
error = indexing_(feature_map_query, torch.flip(p_proj_1,(1,))) - feature_ref
print((error))

tensor([[173, 146],
        [126,  96],
        [100, 155],
        [ 89, 164],
        [155, 158],
        [135, 111],
        [178, 130]], dtype=torch.int32)
tensor([[-115.],
        [ -88.],
        [-132.],
        [-136.],
        [-113.],
        [ -31.],
        [-124.]], dtype=torch.float64)


In [745]:
img = cv2.imread("toy_example/data/toyexample_1.png", 0)
img = img.astype('uint8')
for i, p in enumerate(p_proj_1):
    print(i, p)
    cv2.circle(img, tuple(p.numpy()), 1, (128, 128, 0), -1)
cv2.imshow('image',img)
cv2.waitKey(0)
cv2.destroyAllWindows()
# cv2.imwrite("iter11.png", img)

0 tensor([173, 146], dtype=torch.int32)
1 tensor([127,  96], dtype=torch.int32)
2 tensor([ 99, 154], dtype=torch.int32)
3 tensor([ 88, 162], dtype=torch.int32)
4 tensor([154, 159], dtype=torch.int32)
5 tensor([136, 110], dtype=torch.int32)
6 tensor([178, 130], dtype=torch.int32)


In [760]:
i = 0
verbose = 1
p_3d_1 = torch.mm(R, pts3D.T).T + t
p_proj_1 = torch.round(from_homogeneous(torch.mm(K, p_3d_1.T).T)).type(torch.IntTensor)-1
error = indexing_(feature_map_query, torch.flip(p_proj_1,(1,))) - feature_ref
# error = torch.flip(p_proj_1, (1,)) - ref2d
error = error.type(torch.DoubleTensor)
print(error)
cost = 0.5*(error**2).sum(-1)

if i == 0:
    prev_cost = cost.mean(-1)
if verbose:
    print('Iter ', i, cost.mean().item())

tensor([[-115.],
        [ -88.],
        [-132.],
        [-136.],
        [-113.],
        [ -31.],
        [-124.]], dtype=torch.float64)
Iter  0 6142.5


In [761]:
J_p_T = torch.cat([
    batched_eye_like(p_3d_1, 3), -skew_symmetric(p_3d_1)], -1)

In [762]:
shape = p_3d_1.shape[:-1]
o, z = p_3d_1.new_ones(shape), p_3d_1.new_zeros(shape)
J_e_p = torch.stack([
    K[0,0]*o, z, -K[0,0]*p_3d_1[..., 0] / p_3d_1[..., 2],
    z, K[1,1]*o, -K[1,1]*p_3d_1[..., 1] / p_3d_1[..., 2],
], dim=-1).reshape(shape+(2, 3)) / p_3d_1[..., 2, None, None]

In [763]:
grad_x_points = indexing_(feature_grad_x, torch.flip(p_proj_1,(1,)))
grad_y_points = indexing_(feature_grad_y, torch.flip(p_proj_1,(1,)))
J_p_F = torch.cat((grad_x_points[..., None], grad_y_points[...,None]), -1)

In [764]:
J_e_T = J_p_F @ J_e_p @ J_p_T

In [765]:
Grad = torch.einsum('bij,bi->bj', J_e_T, error)
Grad = Grad.sum(-2)  # Grad was ... x N x 6

In [766]:
J = J_e_T
Hess = torch.einsum('ijk,ijl->ikl', J, J)
# Hess = weights[..., None, None] * Hess
Hess = Hess.sum(-3)  # Hess was ... x N x 6 x 6

In [767]:
torch.matmul(torch.inverse(Hess), Grad[..., None])

tensor([[-2.6382],
        [-8.9607],
        [ 0.2741],
        [-0.0935],
        [ 0.0430],
        [ 0.5951]], dtype=torch.float64)

In [754]:
delta = -0.1*torch.matmul(torch.inverse(Hess), Grad[...,None])[..., 0]

In [755]:
dt, dw = delta[..., :3], delta[..., 3:6]
dr = so3exp_map(dw)
print("dr is : ", dw)
print("dt is : ", dt)
R_new = dr @ R
t_new = dr @ t + dt

dr is :  tensor([ 0.0015, -0.0012, -0.0218], dtype=torch.float64)
dt is :  tensor([ 0.0840,  0.2258, -0.0029], dtype=torch.float64)


In [756]:
print(" R_new ", R_new)
print(" t_new ", t_new)
print("R ", R)
print("t ", t)

 R_new  tensor([[ 9.9777e-01, -6.6773e-02,  5.2216e-04],
        [ 6.6748e-02,  9.9711e-01, -3.6228e-02],
        [ 1.8984e-03,  3.6182e-02,  9.9934e-01]], dtype=torch.float64)
 t_new  tensor([ 0.6018,  1.0098, -0.0339], dtype=torch.float64)
R  tensor([[ 9.9608e-01, -8.8406e-02,  2.4494e-03],
        [ 8.8438e-02,  9.9548e-01, -3.4660e-02],
        [ 6.2580e-04,  3.4740e-02,  9.9940e-01]], dtype=torch.float64)
t  tensor([ 0.5006,  0.7950, -0.0328], dtype=torch.float64)


In [757]:
R, t = R_new, t_new

In [758]:
np.linalg.det(R.numpy())

0.9999999999999999

In [385]:
R_init

tensor([[ 0.9848, -0.1736,  0.0000],
        [ 0.1736,  0.9848,  0.0000],
        [ 0.0000,  0.0000,  1.0000]], dtype=torch.float64)

In [612]:
model = sparse3DBA(n_iters = 500, lambda_ = 0, verbose=True)

In [613]:
R, t = model(pts3D, feature_ref, feature_map_query, feature_grad_x, feature_grad_y, K, R_init, t_init)

Iter  0 11117.142857142857
dr is :  tensor([ 0.0344,  0.0200, -0.0111], dtype=torch.float64)
dt is :  tensor([-0.1927, -0.1188, -0.0185], dtype=torch.float64)
new cost is  17082.214285714286
cost increased, continue with next iteration
Iter  1 11117.142857142857
dr is :  tensor([ 0.0034,  0.0020, -0.0011], dtype=torch.float64)
dt is :  tensor([-0.0193, -0.0119, -0.0019], dtype=torch.float64)
new cost is  11221.0
cost increased, continue with next iteration
Iter  2 11117.142857142857
dr is :  tensor([ 0.0003,  0.0002, -0.0001], dtype=torch.float64)
dt is :  tensor([-0.0019, -0.0012, -0.0002], dtype=torch.float64)
new cost is  10953.214285714286
Iter  3 10953.214285714286
dr is :  tensor([-0.0398, -0.0235, -0.0225], dtype=torch.float64)
dt is :  tensor([ 0.2866,  0.0776, -0.1852], dtype=torch.float64)


IndexError: index 275 is out of bounds for dimension 1 with size 256

In [388]:
R

tensor([[ 9.9970e-01, -2.4542e-02,  1.0347e-04],
        [ 2.4541e-02,  9.9969e-01,  3.6830e-03],
        [-1.9383e-04, -3.6794e-03,  9.9999e-01]], dtype=torch.float64)

In [389]:
t

tensor([-0.0196, -0.0055,  0.0032], dtype=torch.float64)

In [349]:
a @ b

array([[[1.13206942, 1.11894092, 0.57274033, 1.24841041, 0.79335565,
         0.46561771],
        [1.2855546 , 1.19792658, 1.26153542, 1.5414339 , 1.17330447,
         1.04735982]]])